In [1]:
from collections import defaultdict
from random import randint, choice
import operator
alphabets = 'ABCDEFGHI'
digits = range(1,10)

In [2]:
def init_grid():
    d = defaultdict()
    for a in alphabets:
        for ii in digits:
            d[(a,ii)] = 0            
    return d

In [3]:
def peer_grid2(gn):
    e = defaultdict(lambda: defaultdict(list))
    matrix_step = [ (a, ii) for a in ('ABC', 'DEF', 'GHI') for ii in ('123', '456', '789') ]
    matrix2 = defaultdict(int)
    for i, t in enumerate(matrix_step):
        matrix2[t] = i + 1
    for k in gn:
        e[k]['prow'] = [ (k[0], i) for i in digits if i != k[1] ]
        e[k]['pcol'] = [ (a, k[1]) for a in alphabets if a != k[0] ]
        for key in matrix2:
            if k[0] in key[0] and str(k[1]) in key[1]:
                    squarelist = [ (a, int(b)) for a in key[0] for b in key[1] if (a, int(b)) != k]
                    e[k]['psquares'] = [s for s in squarelist ]
    return e

In [4]:
def values_grid2(gn):
    d = defaultdict(int)
    for k in gn:
        if gn[k] != 0:
            d[k] = gn[k]
        if gn[k] == 0:
            d[k] = [ i for i in digits ]
    return d

In [5]:
def eliminate2(gn, pn, vn, r, c):
    peers = []
#     print(gn[(r, c)])
    if type(gn[(r, c)]) == int and gn[(r, c)] != 0:
        for k in pn[(r, c)]:
            for char in pn[(r, c)][k]:
                peers.append(char)
    for peer in peers:
#             print(peer)
#             print(vn[peer])
        if type(vn[peer]) == list:
            if gn[(r, c)] in vn[peer] and len(vn[peer]) != 1:
                vn[peer].remove(gn[(r, c)])
                if len(vn[peer]) == 1:
                    vn[peer] = vn[peer][0]
                    eliminate2(gn, pn, vn, peer[0], peer[1])
    return vn

In [6]:
def bullseye2(gn, pn, vn, r, c):
    nlist = [ vn[vv] for vv in pn[(r, c)]['prow'] ]
    nlist2 = [ vn[vv] for vv in pn[(r, c)]['pcol'] ]
    nlist3 = [ vn[vv] for vv in pn[(r, c)]['psquares'] ]
    num2 = [ n for n in digits if n not in nlist and n not in nlist2 and n not in nlist3]
    if len(num2) == 1: vn[[(r, c)]] = num2
#     print(nlist, nlist2, nlist3, num2)
    eliminate2(gn, pn, vn, r, c)
    return vn

In [7]:
def constraint_prop2(gn, pn, vn, r, c):
    vn = eliminate2(gn, pn, vn, r, c)
    vn = bullseye2(gn, pn, vn, r, c)
    return vn

In [8]:
def search2(gn, pn, vn):
    working = defaultdict(int)
    working = { k: len(v) for k, v in vn if type(v) == list }
    ordering = sorted(working, key=working.get)
    for k in ordering:
        n1, n2 = k[0], k[1]
#         print(vn[k])
        if type(vn[k]) == list:
            start = 0
            if start <= len(vn[k]):
                n3 = vn[k][start]
                square_check = [ k for k in pn[k]['psquares'] if gn[k] != pn[k] ]
                c = all(gn[k] != n3 for k in square_check)
                while r_c_check(vn, n1, n2, n3) is True and c is False:
                    start += 1
                if r_c_check(vn, n1, n2, n3) is False and c is True:
                    vn[k] = n3
                    vn = constraint_prop2(gn, pn, vn, n1, n2)
                    start = 0
    return vn

In [9]:
def random_init(gn, pn, numinit):
    count = 0
    while count <= numinit:
        n1, n2, n3 = choice(alphabets), randint(1,9), randint(1,9)
        if count == 0:
            gn[(n1, n2)] = n3
            count += 1
        if count > 0:
            if r_c_check(gn, n1, n2, n3) is False:
                square_check = [ k for k in pn[(n1, n2)]['psquares'] if gn[k] != pn[k] ]
                if all(gn[k] != n3 for k in square_check):
                    gn[(n1, n2)] = n3
                    count += 1
            else:
                pass  

In [10]:
def r_c_check(gn, r, c, num):
    for i in digits:
        if gn[(r, i)] == num:
            return True
            break
    for a in alphabets:
        if gn[(a, c)] == num:
            return True
            break
    return False

In [11]:
def print_grid(gn):
    for a in alphabets:
        print('  -----------------------------------\n',
            '| {} | {} | {} | {} | {} | {} | {} | {} | {} |'.format(gn[(a, 1)], 
            gn[(a, 2)],
            gn[(a, 3)],
            gn[(a, 4)],
            gn[(a, 5)],
            gn[(a, 6)],
            gn[(a, 7)],
            gn[(a, 8)],
            gn[(a, 9)]))
        if a == 'I':
            print('  -----------------------------------')

In [12]:
def input_grid(gn, x, y, num):
    gn[(x, y)].update(num)
    return gn

In [13]:
def whole_check(gn):
    for k in gn:
        if type(gn[k]) == list:
#         if gn[k] == 0:
            return True
            break
    return False

In [14]:
def solver2(gn, pn, vn):
#     while whole_check(vn) is True:
#             vn = search(vn)
    for i in range(1,10000):
        for k in vn:
            vn = constraint_prop2(gn, pn, vn, k[0], k[1])
            vn = search2(gn, pn, vn)
    return vn

In [15]:
def sudoku():
    d1 = init_grid()
    d2 = peer_grid2(d1)
    random_init(d1, d2, 17)
    d3 = values_grid2(d1)
    return d1, d2, d3

In [16]:
d1, d2, d3 = sudoku()

In [17]:
print_grid(d1)

  -----------------------------------
 | 0 | 0 | 0 | 2 | 0 | 9 | 0 | 3 | 0 |
  -----------------------------------
 | 0 | 0 | 3 | 0 | 0 | 0 | 0 | 0 | 0 |
  -----------------------------------
 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
  -----------------------------------
 | 0 | 4 | 0 | 0 | 0 | 0 | 0 | 0 | 6 |
  -----------------------------------
 | 9 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
  -----------------------------------
 | 2 | 0 | 0 | 0 | 0 | 0 | 0 | 9 | 0 |
  -----------------------------------
 | 8 | 9 | 7 | 0 | 0 | 0 | 0 | 0 | 0 |
  -----------------------------------
 | 3 | 0 | 0 | 9 | 0 | 0 | 0 | 0 | 8 |
  -----------------------------------
 | 0 | 0 | 0 | 0 | 3 | 0 | 6 | 0 | 0 |
  -----------------------------------


In [18]:
print(d2)

defaultdict(<function peer_grid2.<locals>.<lambda> at 0x000001C579858BF8>, {('A', 1): defaultdict(<class 'list'>, {'prow': [('A', 2), ('A', 3), ('A', 4), ('A', 5), ('A', 6), ('A', 7), ('A', 8), ('A', 9)], 'pcol': [('B', 1), ('C', 1), ('D', 1), ('E', 1), ('F', 1), ('G', 1), ('H', 1), ('I', 1)], 'psquares': [('A', 2), ('A', 3), ('B', 1), ('B', 2), ('B', 3), ('C', 1), ('C', 2), ('C', 3)]}), ('A', 2): defaultdict(<class 'list'>, {'prow': [('A', 1), ('A', 3), ('A', 4), ('A', 5), ('A', 6), ('A', 7), ('A', 8), ('A', 9)], 'pcol': [('B', 2), ('C', 2), ('D', 2), ('E', 2), ('F', 2), ('G', 2), ('H', 2), ('I', 2)], 'psquares': [('A', 1), ('A', 3), ('B', 1), ('B', 2), ('B', 3), ('C', 1), ('C', 2), ('C', 3)]}), ('A', 3): defaultdict(<class 'list'>, {'prow': [('A', 1), ('A', 2), ('A', 4), ('A', 5), ('A', 6), ('A', 7), ('A', 8), ('A', 9)], 'pcol': [('B', 3), ('C', 3), ('D', 3), ('E', 3), ('F', 3), ('G', 3), ('H', 3), ('I', 3)], 'psquares': [('A', 1), ('A', 2), ('B', 1), ('B', 2), ('B', 3), ('C', 1), ('

In [19]:
print_grid(d3)

  -----------------------------------
 | [1, 2, 3, 4, 5, 6, 7, 8, 9] | [1, 2, 3, 4, 5, 6, 7, 8, 9] | [1, 2, 3, 4, 5, 6, 7, 8, 9] | 2 | [1, 2, 3, 4, 5, 6, 7, 8, 9] | 9 | [1, 2, 3, 4, 5, 6, 7, 8, 9] | 3 | [1, 2, 3, 4, 5, 6, 7, 8, 9] |
  -----------------------------------
 | [1, 2, 3, 4, 5, 6, 7, 8, 9] | [1, 2, 3, 4, 5, 6, 7, 8, 9] | 3 | [1, 2, 3, 4, 5, 6, 7, 8, 9] | [1, 2, 3, 4, 5, 6, 7, 8, 9] | [1, 2, 3, 4, 5, 6, 7, 8, 9] | [1, 2, 3, 4, 5, 6, 7, 8, 9] | [1, 2, 3, 4, 5, 6, 7, 8, 9] | [1, 2, 3, 4, 5, 6, 7, 8, 9] |
  -----------------------------------
 | [1, 2, 3, 4, 5, 6, 7, 8, 9] | [1, 2, 3, 4, 5, 6, 7, 8, 9] | [1, 2, 3, 4, 5, 6, 7, 8, 9] | [1, 2, 3, 4, 5, 6, 7, 8, 9] | [1, 2, 3, 4, 5, 6, 7, 8, 9] | [1, 2, 3, 4, 5, 6, 7, 8, 9] | [1, 2, 3, 4, 5, 6, 7, 8, 9] | [1, 2, 3, 4, 5, 6, 7, 8, 9] | [1, 2, 3, 4, 5, 6, 7, 8, 9] |
  -----------------------------------
 | [1, 2, 3, 4, 5, 6, 7, 8, 9] | 4 | [1, 2, 3, 4, 5, 6, 7, 8, 9] | [1, 2, 3, 4, 5, 6, 7, 8, 9] | [1, 2, 3, 4, 5, 6, 7, 8, 9] | [1, 2,

In [20]:
print_grid(solver2(d1, d2, d3))

  -----------------------------------
 | [1, 4, 5, 6, 7] | [1, 5, 6, 7, 8] | [1, 4, 5, 6, 8] | 2 | [1, 4, 5, 6, 7, 8] | 9 | [1, 4, 5, 7, 8] | 3 | [1, 4, 5, 7] |
  -----------------------------------
 | [1, 4, 5, 6, 7] | [1, 2, 5, 6, 7, 8] | 3 | [1, 4, 5, 6, 7, 8] | [1, 4, 5, 6, 7, 8] | [1, 4, 5, 6, 7, 8] | [1, 2, 4, 5, 7, 8, 9] | [1, 2, 4, 5, 6, 7, 8] | [1, 2, 4, 5, 7, 9] |
  -----------------------------------
 | [1, 4, 5, 6, 7] | [1, 2, 5, 6, 7, 8] | [1, 2, 4, 5, 6, 8, 9] | [1, 3, 4, 5, 6, 7, 8] | [1, 4, 5, 6, 7, 8] | [1, 3, 4, 5, 6, 7, 8] | [1, 2, 4, 5, 7, 8, 9] | [1, 2, 4, 5, 6, 7, 8] | [1, 2, 4, 5, 7, 9] |
  -----------------------------------
 | [1, 5, 7] | 4 | [1, 5, 8] | [1, 3, 5, 7, 8] | [1, 2, 5, 7, 8, 9] | [1, 2, 3, 5, 7, 8] | [1, 2, 3, 5, 7, 8] | [1, 2, 5, 7, 8] | 6 |
  -----------------------------------
 | 9 | [1, 3, 5, 6, 7, 8] | [1, 5, 6, 8] | [1, 3, 4, 5, 6, 7, 8] | [1, 2, 4, 5, 6, 7, 8] | [1, 2, 3, 4, 5, 6, 7, 8] | [1, 2, 3, 4, 5, 7, 8] | [1, 2, 4, 5, 7, 8] | [1, 2, 3